In [233]:
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
import torch
import pandas as pd

In [3]:
data = pd.read_pickle('data/dataset.pkl')

In [234]:
ques = data.pop('ques')
#data.drop(['ans_id','ans'],axis=1)

NameError: name 'data' is not defined

In [235]:
ques = [
    "Three years later, the coffin was still full of Jello.",
    "The fish dreamed of escaping the fishbowl and into the toilet where he saw his friend go.",
    "The person box was packed with jelly many dozens of months later.",
    "He found a leprechaun in his walnut shell."
    ,"oh man some years later, it was covered Jello."
]

In [254]:
import time

# starting time
start = time.time()

tokenizer = AutoTokenizer.from_pretrained('Models/BERTOverflow')
model = AutoModel.from_pretrained('Models/BERTOverflow')

tokens = {'input_ids': [], 'attention_mask': []}

for q in ques:
    new_tokens = tokenizer.encode_plus(q, max_length=128
                                       , truncation = True
                                       , padding = 'max_length'
                                       , return_tensors ='pt')#pt:pytorch
    
    tokens['input_ids'].append(new_tokens['input_ids'][0])
    tokens['attention_mask'].append(new_tokens['attention_mask'][0])
    
tokens['input_ids'] = torch.stack(tokens['input_ids'])
tokens['attention_mask'] = torch.stack(tokens['attention_mask'])
print(time.time()-start)

10.777753353118896


In [243]:
outputs = model(**tokens)
embeddings = outputs.last_hidden_state

In [244]:
attention_mask = tokens['attention_mask']
attention_mask.shape 

torch.Size([5, 128])

In [245]:
attention_mask = attention_mask.unsqueeze(-1) #add 3rd demension to it
mask=attention_mask.expand(embeddings.shape).float()  #to" expand demension to be like embeddings shape

In [246]:
masked_embeddings = embeddings * mask #remove all masked tokens

In [247]:
summed = torch.sum(masked_embeddings, 1)# 1 here for deminsion want to sum it
summed.shape

torch.Size([5, 768])

In [248]:
counts = torch.clamp(attention_mask.sum(1), min = 1e-9)
counts.shape

torch.Size([5, 1])

In [249]:
mean_pooled = summed / counts
mean_pooled.shape

torch.Size([5, 768])

In [250]:
mean_pooled

tensor([[ 0.1302,  0.3271,  0.4505,  ...,  0.5498, -0.0169,  0.3294],
        [ 0.1905,  0.5644,  0.5213,  ...,  0.8872, -0.0855,  0.2673],
        [ 0.2372,  0.1939,  0.3518,  ...,  1.0909,  0.0673,  0.1922],
        [ 0.2335,  0.3786,  0.5990,  ...,  0.9654, -0.1522,  0.5955],
        [ 0.2979,  0.1022,  0.3277,  ...,  0.7758, -0.0238,  0.2613]],
       grad_fn=<DivBackward0>)

In [251]:
mean_pooled = mean_pooled.detach().numpy()
cosine_similarity(
    [mean_pooled[0]],mean_pooled[1:]
) 

array([[0.96365964, 0.95818937, 0.9498954 , 0.973516  ]], dtype=float32)

In [252]:
0

0